# 練習網址：https://statementdog.com/
## 目標
爬取任一公司(ex. 4754 國碳科)中股票健診的文字，如排除地雷股健診、定存股健診、成長股健診內之完整健診細節。
## 步驟
1. 使用selenium登入會員，得cookies。
2. 將所得cookies塞入requests內，方便後續爬蟲。
3. 因所爬取內容由javascript寫成，故尋找關鍵網址後，開始爬取資料。

In [1]:
#!/usr/bin/env python
#coding=utf-8

In [2]:
# 建立各種 HTTP 請求，從網頁伺服器上取得想要的資料
import requests

# 用來解析處理requests取得的數據; processing XML and HTML
from lxml import html

# 一個HTML解析器，可以處理各種奇怪的HTML
from bs4 import BeautifulSoup as bs

# 模擬人操作行為
from selenium.webdriver import Chrome

# 睡眠用
from time import sleep

# 帳密位置
import sys
sys.path.append("../../../Downloads")
import statementdog

# 處理javascript用
import json

# 能處理載入、整理與視覺化等常見的資料應用套件
import pandas as pd

# 隨機睡眠用
import random

print('套件載入完成')

套件載入完成


#### 在Colab上，取得google drive的存取授權
```python
from google.colab import drive
drive.mount('/content/drive')
```

In [3]:
# 指定driver位置
driver = Chrome('./chromedriver')

In [4]:
# 建立登入相關訊息
# 參考資料：https://ithelp.ithome.com.tw/articles/10193266
LOGIN_URL = 'https://statementdog.com/users/sign_in' #登入會員網址
USERID = statementdog.id() # 帳號
PASSWORD = statementdog.password() # 密碼
HEADERS = statementdog.headers() # headers 

### 使用selenium登入網頁

```python
- driver.find_element_by_id('關鍵字串') # 找id
- driver.find_element_by_class_name('關鍵字串') # 找class
```

In [5]:
driver.get(LOGIN_URL) # 登入會員網址

# 輸入帳號
driver.find_element_by_id('user_email').send_keys(USERID) # 輸入帳號
sleep(3) # 休息3秒

# 輸入密碼
driver.find_element_by_id('user_password').click() # 點擊輸入密碼區域，以便輸入密碼
driver.find_element_by_id('user_password').send_keys(PASSWORD) #輸入密碼

sleep(3) # 休息3秒
driver.find_element_by_class_name('submit-btn').click() # 點擊"登入" # 找無id, 找class

print('登入成功')

登入成功


In [6]:
# 得cookies
cookies_list = driver.get_cookies()

# 關瀏覽器
driver.close()

# 檢視cookies_list
# cookies_list 

### 設定requests

In [7]:
# 請求時保持某些引數，例如cookie
s = requests.session()

# cookies_list為一個list包字典，將cookie名字和值拿出來，然後設定session中
for cookie in cookies_list:
    s.cookies.set(cookie['name'], cookie['value']) # 之後的造訪就會自動帶入cookie

# headers
headers = {'user-agent': HEADERS}

### 爬蟲
因主要爬取之資料由javascript寫成，故需找來源資料。
1. 將selenium登入會員後，所得cookies塞進requests內，方便後續爬蟲。

```python
# 請求時保持某些引數，例如cookie
s = requests.session()

# cookies_list為一個list包字典，將cookie名字和值拿出來，然後設定session中
for cookie in cookies_list:
    s.cookies.set(cookie['name'], cookie['value']) # 之後的造訪就會自動帶入cookie
```

2. 排除「地雷股健診」、「定存股健診」、「成長股健診」報告敘述，找到來源網址後解析(程式碼如下)，發現主要報告內容為計算及判別式所編成，故寫成函式後續利用：

```python
# 從Network > JS > analysis-50... > Headers 之Request URL找到引用網址，透過Response含所需資料
report = s.get('https://statementdog.com/assets/analysis-a50d1de8003ac1709244712dde20436832c4c8549f911dba16e94febae9a62e5.js', headers = headers)
reprotText = report.text

# 將報告轉碼至人類可讀，可發現主要報告內容為計算及判別式所編成
reprotText = reprotText.encode('utf-8').decode('unicode_escape')
reprotText

# 寫成函式，方便排除地雷股健診、定存股健診、成長股健診報告呈現利用。
```

3. 「完整健診細節」內容，找到來源網址後，發現為json格式，直接可以利用。

- **報告內容之函式**

In [8]:
# 從Network > JS > analysis-50... > Headers 之Request URL找到引用網址，透過Response含所需資料
report = s.get('https://statementdog.com/assets/analysis-a50d1de8003ac1709244712dde20436832c4c8549f911dba16e94febae9a62e5.js', headers = headers)
reprotText = report.text

# 將報告轉碼至人類可讀，可發現主要報告內容為計算及判別式所編成
reprotText = reprotText.encode('utf-8').decode('unicode_escape')
reprotText

/Users/TAI/.local/share/virtualenvs/WebCrawler-Q8hI92k6/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: invalid escape sequence '\d'
  


'/*\nCopyright (c) 2010,2011,2012,2013,2014 Morgan Roderick http://roderick.dk\nLicense: MIT - http://mrgnrdrck.mit-license.org\n\nhttps://github.com/mroderick/PubSubJS\n*/\nfunction stopPropagation(e){e.stopPropagation?e.stopPropagation():window.event&&(window.event.cancelBubble=!0)}function detectDevice(){var e=window.innerWidth>0?window.innerWidth:screen.width;return e<=568?"mobile":e<=768?"pad":"desktop"}function initCommonVar(){gVar.stockNm="",gVar.trackStockNum=10,gVar.trackGroupNum=5,gVar.socket=null,gVar.rooms=[],gVar.stream={},gVar.stream.data=null,gVar.stream.oldPtr=null,gVar.stream.lastPtr=null}function clearStockId(e){$(e).val("")}function stockIdTip(e){""==$(e).val()&&$(e).val("輸入股號 / 股名")}function stockQueryKeypress(e){window.event&&"13"==e.keyCode?$(".stock-search-btn").trigger("click"):e.which&&"13"==e.which&&$(".stock-search-btn").trigger("click")}function jumpToPage(e){var t="/analysis/tpe/"+encodeURIComponent($("#stockid").val());e?window.open(t):location=t}function 

In [9]:
# 將報告寫成函式
# 排除地雷股健診
def bombReport_(name, passedRatio):
    if passedRatio <= 40:
        bombText = '{name}通過 {passedRatio}% 排除地雷股檢查項目，顯示公司財務體質風險高。如果你偏好體質穩健的公司，{name}可以先暫時觀望。'
    elif 40 < passedRatio <50:  
        bombText = '{name}通過 {passedRatio}% 排除地雷股檢查項目，代表公司財務體質有部分缺陷，沒有通過的檢查項目都必須特別留意小心。'
    elif 50 <= passedRatio <67:  
        bombText = '{name}通過 {passedRatio}% 排除地雷股檢查項目，整體財務體質表現算及格。如果其他財務健診結果也不錯，可以考慮加入追蹤觀察。'
    elif passedRatio >= 67:
        bombText = '{name}通過 {passedRatio}% 排除地雷股檢查項目，代表公司是地雷股的風險低。如果你偏好體質穩健的公司，{name}值得你加入追蹤觀察。'
    bombReport = bombText.format(name=name, passedRatio=passedRatio)
    return bombReport

# 定存股健診
def cd_(name, passedRatio):
    if passedRatio <= 40:
        cdText = '{name}通過 {passedRatio}% 定存股鑑定項目，顯示公司股利發放與殖利率表現差。如果你偏好高股息高殖利率的公司，{name}並不適合你。'
    elif 40 < passedRatio <50:  
        cdText = '{name}通過 {passedRatio}% 定存股鑑定項目，顯示公司股利發放與殖利率表現不吸引人。如果你偏好高股息高殖利率的公司，{name}可以先暫時觀望'
    elif 50 <= passedRatio <67:  
        cdText = '{name}通過 {passedRatio}% 定存股鑑定項目，公司股利發放與殖利率表現還算及格。如果其他財務健診結果也不錯，可以考慮加入追蹤觀察。'
    elif passedRatio >= 67:
        cdText = '{name}通過 {passedRatio}% 定存股鑑定項目，代表公司股利發放與殖利率表現優異。如果你偏好高股息高殖利率的公司，",i,"值得你加入追蹤觀察。'
    cdReport = cdText.format(name=name, passedRatio=passedRatio)
    return cdReport

# 成長股健診
def growth_(name, passedRatio):
    if passedRatio <= 40:
        growthText = '{name}通過 {passedRatio}% 成長股檢查項目，顯示公司近期業績成長表現差勁。如果你偏好短期有成長動能的公司，{name}並不適合你。'
    elif 40 < passedRatio <50:  
        growthText = '{name}通過 {passedRatio}% 成長股檢查項目，顯示公司近期業績成長表現不吸引人。如果你偏好短期有成長動能的公司，{name}可先暫時觀望。'
    elif 50 <= passedRatio <67:  
        growthText = '{name}通過 {passedRatio}% 成長股檢查項目，代表近期成長表現還算及格。如果其他財務健診結果也不錯，可以考慮加入追蹤觀察。'
    elif passedRatio >= 67:
        growthText = '{name}通過 {passedRatio}% 成長股檢查項目，代表公司近期業績成長優異。如果你偏好短期有成長動能的公司，{name}值得你加入追蹤觀察。'
    growthReport = growthText.format(name=name, passedRatio=passedRatio)
    return growthReport

# 便宜股健診
def cheap_(name, passedRatio):
    if passedRatio <= 40:
        cheapText = '{name}通過 {passedRatio}% 股價是否低估的檢查項目，初步來看公司股價並未低估。如果你偏好股價有明顯低估的公司，{name}並不適合你。'
    elif 40 < passedRatio <50:  
        cheapText = '{name}通過 {passedRatio}% 股價是否低估的檢查項目，初步來看公司股價未明顯低估。除非其他財務表現都不錯，不然{name}可先暫時觀望。'
    elif 50 <= passedRatio <67:  
        cheapText = '{name}通過 {passedRatio}% 股價是否低估的檢查項目，初步來看公司股價並不算貴。如果其他財務健診結果也不錯，{name}可以考慮進一步深入研究。'
    elif passedRatio >= 67:
        cheapText = '{name}通過 {passedRatio}%股價是否低估的檢查項目，顯示股價可能被低估。如果你偏好股價低估的公司，{name}可以加入追蹤觀察。'
    cheapReport = cheapText.format(name=name, passedRatio=passedRatio)
    return cheapReport

- **「完整健診細節」內容**
    - 先以4754 國碳科之「排除地雷股健診」、「定存股健診」、「成長股健診」細節為範例。

In [10]:
# 從Network > XHR > healthcheck > Headers之Request URL找到引用網址，透過Response含所需資料
apiURL = 'https://statementdog.com/api/v1/stocks/'
res = s.get(apiURL  + '4754' + '/health_check', headers = headers)
res.text # 得資料，發現為json

'{"bomb":{"status":"confirmed","check_list":{"count":6,"passed":1,"pass_ratio":17.0,"items":[{"name":"自由現金流入近五年有三年大於 0","value":true},{"name":"自由現金流入近五年平均大於 0","value":false},{"name":"營業現金流入對淨利比近五年有三年大於 100%","value":false},{"name":"營業現金流入對淨利比近五年平均大於 100%","value":false},{"name":"應收帳款週轉天數小於等於去年同期數據","value":false},{"name":"存貨週轉天數小於等於去年同期數據","value":false}]},"task":null},"cd":{"status":"confirmed","check_list":{"count":5,"passed":3,"pass_ratio":60.0,"items":[{"name":"近一年股息殖利率大於 6 %","value":false},{"name":"近五年平均股息殖利率大於 6 %","value":false},{"name":"連續五年都有發股息","value":true},{"name":"股息發放率五年內有三年大於 50 %","value":true},{"name":"股息發放率五年平均大於 50%","value":true}]},"task":"註冊成爲免費會員"},"growth":{"status":"confirmed","check_list":{"count":5,"passed":1,"pass_ratio":20.0,"items":[{"name":"月營收 YOY 連續三個月大於 0","value":false},{"name":"近一季毛利年增率大於 0","value":true},{"name":"近一季營業利益年增率大於 0","value":false},{"name":"近一季稅前淨利年增率大於 0","value":false},{"name":"近一季稅後淨利年增率大於 0","value":false}]},"task":"註冊成爲免費會員"},"che

In [11]:
# 整理成json格式，人類可讀
data = json.loads(res.text)
data

{'bomb': {'status': 'confirmed',
  'check_list': {'count': 6,
   'passed': 1,
   'pass_ratio': 17.0,
   'items': [{'name': '自由現金流入近五年有三年大於 0', 'value': True},
    {'name': '自由現金流入近五年平均大於 0', 'value': False},
    {'name': '營業現金流入對淨利比近五年有三年大於 100%', 'value': False},
    {'name': '營業現金流入對淨利比近五年平均大於 100%', 'value': False},
    {'name': '應收帳款週轉天數小於等於去年同期數據', 'value': False},
    {'name': '存貨週轉天數小於等於去年同期數據', 'value': False}]},
  'task': None},
 'cd': {'status': 'confirmed',
  'check_list': {'count': 5,
   'passed': 3,
   'pass_ratio': 60.0,
   'items': [{'name': '近一年股息殖利率大於 6 %', 'value': False},
    {'name': '近五年平均股息殖利率大於 6 %', 'value': False},
    {'name': '連續五年都有發股息', 'value': True},
    {'name': '股息發放率五年內有三年大於 50 %', 'value': True},
    {'name': '股息發放率五年平均大於 50%', 'value': True}]},
  'task': '註冊成爲免費會員'},
 'growth': {'status': 'confirmed',
  'check_list': {'count': 5,
   'passed': 1,
   'pass_ratio': 20.0,
   'items': [{'name': '月營收 YOY 連續三個月大於 0', 'value': False},
    {'name': '近一季毛利年增率大

In [12]:
# 找出jason的key值
data.keys()

dict_keys(['bomb', 'cd', 'growth', 'cheap', 'chip', 'quality', 'turnaround', 'stock', 'total_modules_count', 'unlocked_modules_count', 'category'])

其中'bomb'為「排除地雷股健診」、'cd'為「定存股健診」、'growth'為「成長股健診」

In [13]:
# 讀取之前下載好的有有價證券代號及名稱
stockList = pd.read_csv("stockList.csv", encoding='utf-8')
stockList.tail(5)

,有價證券代號及名稱,國際證券辨識號碼(ISIN Code),上市日,市場別,產業別,CFICode,備註
937,9944 新麗,TW0009944009,2002/8/26,上市,其他業,ESVUFR,NaN
938,9945 潤泰新,TW0009945006,1992/4/30,上市,其他業,ESVUFR,NaN
939,9946 三發地產,TW0009946004,2013/9/17,上市,建材營造業,ESVUFR,NaN
940,9955 佳龍,TW0009955005,2008/1/21,上市,其他業,ESVUFR,NaN
941,9958 世紀鋼,TW0009958009,2008/3/12,上市,鋼鐵工業,ESVUFR,NaN


In [14]:
# 僅需要'有價證券代號及名稱'
# 檢視其中一個代號及名稱
stockList = stockList['有價證券代號及名稱'] 
stockList

0        1101　台泥
1        1102　亞泥
2        1103　嘉泥
3        1104　環泥
4        1108　幸福
         ...    
937      9944　新麗
938     9945　潤泰新
939    9946　三發地產
940      9955　佳龍
941     9958　世紀鋼
Name: 有價證券代號及名稱, Length: 942, dtype: object

In [15]:
# 檢視一下其中一項
stockList[0]

'1101\u3000台泥'

發現有全行空白，記得避開。

In [16]:
# 範例
num = stockList[0][:4] # 有價證券代號
name = stockList[0][5:] # 有價證券名稱
print('有價證券代號:', num)
print('有價證券名稱:', name)

有價證券代號: 1101
有價證券名稱: 台泥


### 開始爬蟲

In [ ]:
# 設個最終字串，供爬完後放入
final_dic = []
error_list = []

for stock in stockList:
    try:
        print('開始抓取', stock, "之資料")

        # 有價證券代號及名稱
        num = stock[:4] # 有價證券代號
        name = stock[5:] # 有價證券名稱
        print('設定有價證券代號及名稱: 代號:{num}, 名稱:{name}'.format(num=num, name=name))

        # 健診細節之api網址
        apiURL = 'https://statementdog.com/api/v1/stocks/' + str(num) + '/health_check'
        res = s.get(apiURL, headers = headers)
        print('設定', stock, ", api網址:" + apiURL)
        
        # 將健診細節轉換成json檔
        data = json.loads(res.text)

        # 擷取json檔，設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」通過條件比例
        bomb_passedRatio = data['bomb']['check_list']['pass_ratio']
        cd_passedRatio = data['cd']['check_list']['pass_ratio']
        growth_passedRatio = data['growth']['check_list']['pass_ratio']
        cheap_passedRatio = data['cheap']['check_list']['pass_ratio']
        print('設定健檢通過比例參數:「排除地雷股健診」: {bomb_passedRatio}%,「定存股健診」: {cd_passedRatio}%,「成長股健診」: {growth_passedRatio}%,「便宜股健診」: {cheap_passedRatio}%'.format(bomb_passedRatio=bomb_passedRatio, cd_passedRatio=cd_passedRatio, growth_passedRatio=growth_passedRatio, cheap_passedRatio=cheap_passedRatio))

        # 設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
        bomb = data['bomb']['check_list']['items']
        cd = data['cd']['check_list']['items']
        growth = data['growth']['check_list']['items']
        cheap = data['cheap']['check_list']['items']
        print('設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節')

        print('建立該股字典')
        # 創造空字典，供建立該股報告
        dic= {'有價證券代號':num,
              '有價證券名稱':name,
              '排除地雷股健診':bombReport_(name, bomb_passedRatio),
              '通過排除地雷股條件比例(%)':bomb_passedRatio,
              bomb[0]['name']:bomb[0]['value'],
              bomb[1]['name']:bomb[1]['value'],
              bomb[2]['name']:bomb[2]['value'],
              bomb[3]['name']:bomb[3]['value'],
              bomb[4]['name']:bomb[4]['value'],
              bomb[5]['name']:bomb[5]['value'],

              '定存股健診':cd_(name, cd_passedRatio),
              '通過定存股健診條件比例(%)':cd_passedRatio,
              cd[0]['name']:cd[0]['value'],
              cd[1]['name']:cd[1]['value'],
              cd[2]['name']:cd[2]['value'],
              cd[3]['name']:cd[3]['value'],
              cd[4]['name']:cd[4]['value'],

              '成長股健診':growth_(name, growth_passedRatio),
              '通過成長股健診條件比例(%)':growth_passedRatio,
              growth[0]['name']:growth[0]['value'],
              growth[1]['name']:growth[1]['value'],
              growth[2]['name']:growth[2]['value'],
              growth[3]['name']:growth[3]['value'],
              growth[4]['name']:growth[4]['value'], 
              
              '便宜股健診':cheap_(name, cheap_passedRatio),
              '通過便宜股健診條件比例(%)':cheap_passedRatio,
              cheap[0]['name']:cheap[0]['value'],
              cheap[1]['name']:cheap[1]['value'],
              cheap[2]['name']:cheap[2]['value'],
              cheap[3]['name']:cheap[3]['value'],
              cheap[4]['name']:cheap[4]['value'] 
        } 

        print(name + "加入總字典")
        print("-"*80)
        final_dic.append(dic)

        # 隨機秒數休眠
        sleepTime = random.randint(1, 5) 
        sleep(sleepTime)
        
    except:
        error_list.append[stock] # 將沒搜尋到證劵集中，方便後續檢視
    
print("-"*80)
print('全部爬取完成')

開始抓取 1101　台泥 之資料
設定有價證券代號及名稱: 代號:1101, 名稱:台泥
設定 1101　台泥 , api網址:https://statementdog.com/api/v1/stocks/1101/health_check
設定健檢通過比例參數:「排除地雷股健診」: 67.0%,「定存股健診」: 80.0%,「成長股健診」: 80.0%,「便宜股健診」: 67.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
台泥加入總字典
--------------------------------------------------------------------------------
開始抓取 1102　亞泥 之資料
設定有價證券代號及名稱: 代號:1102, 名稱:亞泥
設定 1102　亞泥 , api網址:https://statementdog.com/api/v1/stocks/1102/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 60.0%,「成長股健診」: 80.0%,「便宜股健診」: 50.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
亞泥加入總字典
--------------------------------------------------------------------------------
開始抓取 1103　嘉泥 之資料
設定有價證券代號及名稱: 代號:1103, 名稱:嘉泥
設定 1103　嘉泥 , api網址:https://statementdog.com/api/v1/stocks/1103/health_check
設定健檢通過比例參數:「排除地雷股健診」: 33.0%,「定存股健診」: 60.0%,「成長股健診」: 40.0%,「便宜股健診」: 33.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
嘉泥加入總字典
--------------------------------------------------------------------------------
開始抓取 1104　環泥 之資料

開始抓取 1236　宏亞 之資料
設定有價證券代號及名稱: 代號:1236, 名稱:宏亞
設定 1236　宏亞 , api網址:https://statementdog.com/api/v1/stocks/1236/health_check
設定健檢通過比例參數:「排除地雷股健診」: 67.0%,「定存股健診」: 60.0%,「成長股健診」: 20.0%,「便宜股健診」: 17.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
宏亞加入總字典
--------------------------------------------------------------------------------
開始抓取 1256　鮮活果汁-KY 之資料
設定有價證券代號及名稱: 代號:1256, 名稱:鮮活果汁-KY
設定 1256　鮮活果汁-KY , api網址:https://statementdog.com/api/v1/stocks/1256/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 60.0%,「成長股健診」: 100.0%,「便宜股健診」: 17.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
鮮活果汁-KY加入總字典
--------------------------------------------------------------------------------
開始抓取 1301　台塑 之資料
設定有價證券代號及名稱: 代號:1301, 名稱:台塑
設定 1301　台塑 , api網址:https://statementdog.com/api/v1/stocks/1301/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 60.0%,「成長股健診」: 0.0%,「便宜股健診」: 0.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
台塑加入總字典
------------------------------------------------------------------------------

開始抓取 1341　富林-KY 之資料
設定有價證券代號及名稱: 代號:1341, 名稱:富林-KY
設定 1341　富林-KY , api網址:https://statementdog.com/api/v1/stocks/1341/health_check
設定健檢通過比例參數:「排除地雷股健診」: 0.0%,「定存股健診」: 0.0%,「成長股健診」: 80.0%,「便宜股健診」: 17.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
富林-KY加入總字典
--------------------------------------------------------------------------------
開始抓取 1402　遠東新 之資料
設定有價證券代號及名稱: 代號:1402, 名稱:遠東新
設定 1402　遠東新 , api網址:https://statementdog.com/api/v1/stocks/1402/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 60.0%,「成長股健診」: 0.0%,「便宜股健診」: 17.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
遠東新加入總字典
--------------------------------------------------------------------------------
開始抓取 1409　新纖 之資料
設定有價證券代號及名稱: 代號:1409, 名稱:新纖
設定 1409　新纖 , api網址:https://statementdog.com/api/v1/stocks/1409/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 80.0%,「成長股健診」: 0.0%,「便宜股健診」: 50.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
新纖加入總字典
--------------------------------------------------------------------------------
開始抓取

開始抓取 1449　佳和 之資料
設定有價證券代號及名稱: 代號:1449, 名稱:佳和
設定 1449　佳和 , api網址:https://statementdog.com/api/v1/stocks/1449/health_check
設定健檢通過比例參數:「排除地雷股健診」: 33.0%,「定存股健診」: 0.0%,「成長股健診」: 20.0%,「便宜股健診」: 17.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
佳和加入總字典
--------------------------------------------------------------------------------
開始抓取 1451　年興 之資料
設定有價證券代號及名稱: 代號:1451, 名稱:年興
設定 1451　年興 , api網址:https://statementdog.com/api/v1/stocks/1451/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 100.0%,「成長股健診」: 0.0%,「便宜股健診」: 67.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
年興加入總字典
--------------------------------------------------------------------------------
開始抓取 1452　宏益 之資料
設定有價證券代號及名稱: 代號:1452, 名稱:宏益
設定 1452　宏益 , api網址:https://statementdog.com/api/v1/stocks/1452/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 100.0%,「成長股健診」: 0.0%,「便宜股健診」: 67.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
宏益加入總字典
--------------------------------------------------------------------------------
開始抓取 1453　大將 之資料


開始抓取 1506　正道 之資料
設定有價證券代號及名稱: 代號:1506, 名稱:正道
設定 1506　正道 , api網址:https://statementdog.com/api/v1/stocks/1506/health_check
設定健檢通過比例參數:「排除地雷股健診」: 17.0%,「定存股健診」: 0.0%,「成長股健診」: 0.0%,「便宜股健診」: 0.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
正道加入總字典
--------------------------------------------------------------------------------
開始抓取 1507　永大 之資料
設定有價證券代號及名稱: 代號:1507, 名稱:永大
設定 1507　永大 , api網址:https://statementdog.com/api/v1/stocks/1507/health_check
設定健檢通過比例參數:「排除地雷股健診」: 83.0%,「定存股健診」: 60.0%,「成長股健診」: 80.0%,「便宜股健診」: 0.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
永大加入總字典
--------------------------------------------------------------------------------
開始抓取 1512　瑞利 之資料
設定有價證券代號及名稱: 代號:1512, 名稱:瑞利
設定 1512　瑞利 , api網址:https://statementdog.com/api/v1/stocks/1512/health_check
設定健檢通過比例參數:「排除地雷股健診」: 50.0%,「定存股健診」: 0.0%,「成長股健診」: 0.0%,「便宜股健診」: 0.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
瑞利加入總字典
--------------------------------------------------------------------------------
開始抓取 1513　中興電 之資料
設定有價證

開始抓取 1539　巨庭 之資料
設定有價證券代號及名稱: 代號:1539, 名稱:巨庭
設定 1539　巨庭 , api網址:https://statementdog.com/api/v1/stocks/1539/health_check
設定健檢通過比例參數:「排除地雷股健診」: 50.0%,「定存股健診」: 60.0%,「成長股健診」: 60.0%,「便宜股健診」: 100.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
巨庭加入總字典
--------------------------------------------------------------------------------
開始抓取 1540　喬福 之資料
設定有價證券代號及名稱: 代號:1540, 名稱:喬福
設定 1540　喬福 , api網址:https://statementdog.com/api/v1/stocks/1540/health_check
設定健檢通過比例參數:「排除地雷股健診」: 50.0%,「定存股健診」: 80.0%,「成長股健診」: 40.0%,「便宜股健診」: 67.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
喬福加入總字典
--------------------------------------------------------------------------------
開始抓取 1541　錩泰 之資料
設定有價證券代號及名稱: 代號:1541, 名稱:錩泰
設定 1541　錩泰 , api網址:https://statementdog.com/api/v1/stocks/1541/health_check
設定健檢通過比例參數:「排除地雷股健診」: 50.0%,「定存股健診」: 60.0%,「成長股健診」: 80.0%,「便宜股健診」: 0.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
錩泰加入總字典
--------------------------------------------------------------------------------
開始抓取 1558　伸興 之資料

開始抓取 1626　艾美特-KY 之資料
設定有價證券代號及名稱: 代號:1626, 名稱:艾美特-KY
設定 1626　艾美特-KY , api網址:https://statementdog.com/api/v1/stocks/1626/health_check
設定健檢通過比例參數:「排除地雷股健診」: 67.0%,「定存股健診」: 20.0%,「成長股健診」: 20.0%,「便宜股健診」: 50.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
艾美特-KY加入總字典
--------------------------------------------------------------------------------
開始抓取 1701　中化 之資料
設定有價證券代號及名稱: 代號:1701, 名稱:中化
設定 1701　中化 , api網址:https://statementdog.com/api/v1/stocks/1701/health_check
設定健檢通過比例參數:「排除地雷股健診」: 50.0%,「定存股健診」: 60.0%,「成長股健診」: 80.0%,「便宜股健診」: 33.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
中化加入總字典
--------------------------------------------------------------------------------
開始抓取 1702　南僑 之資料
設定有價證券代號及名稱: 代號:1702, 名稱:南僑
設定 1702　南僑 , api網址:https://statementdog.com/api/v1/stocks/1702/health_check
設定健檢通過比例參數:「排除地雷股健診」: 100.0%,「定存股健診」: 60.0%,「成長股健診」: 60.0%,「便宜股健診」: 50.0%
設定「排除地雷股健診」、「定存股健診」、「成長股健診」、「便宜股健診」細節
建立該股字典
南僑加入總字典
--------------------------------------------------------------------------------

In [ ]:
# 檢視錯誤搜尋(沒搜尋到證劵代號)
error_list

### 存擋

In [ ]:
## 存檔

print("存檔中...") #檢查哨
print("")

dn = "ouptput" # 設立output資料夾
file = "stock_statementdog" # 檔名

try:
    if not os.path.exists(dn):   
        os.makedirs(dn)
except:
    pass
    
# 存json格式
f = open(dn + "/"+ file + ".json", "w", encoding="utf-8") # 打開路徑
json.dump(final_dic, f) # 寫入json格式
f.close() # 關閉路徑
print("json存檔完成")
print("")

# 存csv格式
final_dic_df = pd.DataFrame(final_dic)
final_dic_df.to_csv(dn + "/"+ file + ".csv",encoding="utf-8", index=False)

print("csv存檔完成")
print("")

# # DONE
print("存檔完成")
print("="*90)

print("Done!")

In [ ]:
# 檢視一下資料
final_dic_df.head(5)

In [ ]:
final_dic_df_fillna = final_dic_df.fillna('False')
final_dic_df_fillna.head(5)

In [ ]:
# 儲存fillna('False')之資料
file_fillna = "stock_statementdog_fillna"
final_dic_df_fillna.to_csv(dn + "/"+ file_fillna + ".csv",encoding="utf-8", index=False)